<a href="https://colab.research.google.com/github/kayquemelo/front/blob/main/fundsexplorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y
!pip install selenium
!pip install webdriver-manager


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [910 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,119 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,218 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,396 kB]


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

def fetch_html(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.binary_location = "/usr/bin/google-chrome"

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    try:
        driver.get(url)

        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )

        html = driver.page_source
        return html
    except Exception as e:
        print(f"Erro ao carregar a página com Selenium: {e}")
        return None
    finally:
        driver.quit()

def parse_table(html, table_index=0):
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all('table')

    if not tables or table_index >= len(tables):
        print(f"Tabela com índice {table_index} não encontrada.")
        return None

    table = tables[table_index]

    headers = [th.text.strip() for th in table.find_all('th')]

    rows = []
    for tr in table.find_all('tr')[1:]:
        cells = [cell.text.strip() for cell in tr.find_all('td')]
        rows.append(cells)

    return headers, rows

def save_to_file(headers, rows, filename, file_format='xlsx'):
    df = pd.DataFrame(rows, columns=headers)

    if file_format.lower() == 'xlsx':
        df.to_excel(filename + "." + file_format, index=False)
    elif file_format.lower() == 'csv':
        df.to_csv(filename + "." + file_format, index=False)
    else:
        print("Use 'xlsx' ou 'csv'.")

def main(url, filename, table_index=0, file_format='excel'):
    html = fetch_html(url)

    if html:
        headers, rows = parse_table(html, table_index)

        if headers and rows:
            save_to_file(headers, rows, filename, file_format)
            print(f"Dados salvos com sucesso em {filename}!")
        else:
            print("Não foi possível extrair dados da tabela.")
    else:
        print("Falha ao obter HTML da página.")

if __name__ == "__main__":
    URL = 'https://www.fundsexplorer.com.br/ranking'
    FILENAME = "scraping"
    TABLE_INDEX = 0
    FILE_FORMAT = 'csv'

    main(URL, FILENAME, TABLE_INDEX, FILE_FORMAT)


Dados salvos com sucesso em scraping20240806!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')